In [ ]:
Sectors = list(Combined_Data['sector'].unique())
Total_Sector_Summary = Group_Consolidator(
    Combined_Data = Combined_Data,
    groups = Sectors,
    column = 'sector'
)
Sector_Data = Group_Consolidator(Combined_Data = Combined_Data,
                                    groups = Sectors,
                                    column = 'sector',
                                    q = 0.20
)
if len(Sector_Data) == 0:
    Sector_Data = list()
    print("No Viable Sectors Currently")

In [ ]:
Sector_Summary = Sector_Data. \
    drop(['last_period_return','last_price','mu_day_up','sd_day_up','mu_day_down','sd_day_down','sd_ret','sd_loss'],axis = 1)
    
Sector_Summary

In [ ]:
if type(Sector_Summary) is not list:
    Top_Sector = Combined_Data[Combined_Data['sector'].isin(Sector_Summary.index.values)]
    Industries = list(Top_Sector['industry'].unique())
    Industry_Data = Group_Consolidator(Combined_Data = Top_Sector,
                                          groups = Industries,
                                          column = 'industry',
                                          q = 0.20
                                         )
else:
    print("No sectors to loop through")
    Industry_Data = list()
    

In [ ]:
Industry_Summary = Industry_Data. \
    drop(['mu_day_up','sd_day_up','mu_day_down','sd_day_down'],axis = 1). \
    query('quant_day_up > 0 & quant_day_down < 0'). \
    query('macd > 0 & risk_ratio > 1')
Industry_Summary

In [ ]:
if type(Industry_Summary) is not list:
    Top_Industry = Top_Sector[Top_Sector['industry'].isin(Industry_Summary.index.values)]
    Stocks = list(Top_Industry['stock'].unique())
#     Stocks = ['SQQQ','TQQQ']
    Stock_Data = Group_Consolidator(Combined_Data = Top_Industry,
                                       groups = Stocks,
                                       column = 'stock',
                                       q = 0.20
    )
else:
    print("No industries to loop through")
    Stock_Data = list()

In [ ]:
Stock_Summary = Stock_Data. \
    drop(['mu_day_up','sd_day_up','mu_day_down','sd_day_down'],axis = 1). \
    query('quant_day_up > 0 & quant_day_down < 0'). \
    query('macd > 0 & alpha > 0 & alpha_p < 0.25')

Stock_Summary

In [ ]:
for Account in ['app','rh']:
    ## Pulling Current Stock Holdings
    Current_Holdings = Get_Holdings(Account)
    Current_Holdings = []

    ## Combining Lists
    if isinstance(Current_Holdings,pd.DataFrame):
        if Current_Holdings.empty:
            Stocks = []
        else:
            Stocks = list(Current_Holdings.keys())
    else:
        if not Current_Holdings:
            Stocks = []
        else:
            Stocks = [i.symbol for i in Current_Holdings]

    if len(Stock_Summary) != 0:
        Stocks.extend(list(Stock_Summary.index))
    Stocks = list(set(Stocks))

    ## Pulling Historical Data
    Hist_Data = pd.DataFrame()
    for Stock in Stocks:
        Hist_Data[Stock] = Combined_Data['close'][Combined_Data.stock == Stock].tail(OLS_Window*5)

    Market_Data = pd.DataFrame({'mkt':Total_Market.loc[:,'close'].tail(OLS_Window*5)})

    ## Creating expected mean and variance matricies
    mu = expected_returns.capm_return(Hist_Data,Market_Data)
    S = risk_models.exp_cov(Hist_Data) #Get the sample covariance matrix

    ## Optimizing Sharpe Ratio
    ef = EfficientFrontier(
        expected_returns = mu,
        cov_matrix = S, 
        weight_bounds = (0,0.2),
        verbose = False
    )
    # weights = ef.max_sharpe()
    weights = ef.min_volatility()
    cleaned_weights = ef.clean_weights(cutoff = 0.01) 

    ## Subsetting Stock Picks
    Final_Picks = []
    for stock in cleaned_weights.keys():
        if cleaned_weights[stock] != 0:
            print(stock," : ",cleaned_weights[stock])
            Final_Picks.append(stock)

    Optimized_Portfolio = ef.portfolio_performance(verbose=True)
        ## Reading In Functions
    Function_Files = os.listdir('Functions')
    for File in Function_Files:
        if '.py' in File:
            print(File)
            Full_File = 'Functions/'+File
            exec(open(Full_File).read())

    ## Quantile for Deciding Limit Orders
    q = 0.20
    ## Quantile for Deciding Stop Loss / Take Profit Orders
    qsp = 0.80

    ## Pulling Relevent Account Information
    Account_Equity = Get_Equity(Account)

    ## Pulling Open Orders
    Open_Orders = Get_Open_Orders(Account)

    ## Pulling Current Stock Holdings
    Current_Holdings = Get_Holdings(Account)

    ## Checking Portfolio Balancing And Placing New Orders
    for s in Final_Picks:

        # Optimal Portfolio Percentage
        Rcm_Holding = cleaned_weights[s]*100

        # Portfolio Restructuring
        if s in list(Current_Holdings.keys()):

            # Current Portfolio Percentage
            Pct_Holding = np.round(float(Current_Holdings[s].equity)*100/Account_Equity,2)
            Pct_Return = float(Current_Holdings[s]['percent_change'])

            # Rebalancing If Beyond Recommended Holding
            if Pct_Holding > Rcm_Holding*1.05:
                Rebalance_Lower(s,q,Account)

            # Checking If More Needs Purchased
            elif Pct_Holding < Rcm_Holding*0.95:
                Rebalance_Higher(s,q,Account)

            # Make No Changes    
            else:
                print("\n",s," Within Recommended Percentage")
                Exit_Orders(s,q = qsp,Account = Account)        

        ## Stocks Not Currently Held
        else:
            Open_Position(
                s = s,
                Rcm_Holding = Rcm_Holding,
                q = q,
                Account = Account)

    ## Closing Required Positions
    if Current_Holdings.empty:
         print("\nNo positions to close, none are held")
    else:
        for s in [s for s in list(Current_Holdings.keys()) if s not in Final_Picks]:
            Close_Position(s,Account)